# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=4)

### Fit a Model

Fit a linear regression model on the training set

In [4]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [5]:
# Instantiate and fit a linear regression model
linreg = LinearRegression()
    
# Fit the model on train data
linreg.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [6]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error

In [7]:
# Calculate MSE on test set
train_mse = mean_squared_error(y_train, linreg.predict(X_train))
test_mse = mean_squared_error(y_test, linreg.predict(X_test))

print('Train Mean Squared Error:', train_mse)
print('Test Mean Squared Error:', test_mse)

Train Mean Squared Error: 0.1654390507077781
Test Mean Squared Error: 0.15233997210708167


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [8]:
# Import cross_val_score from sklearn.model_selection
from sklearn.model_selection import cross_val_score

In [9]:
# Find MSE scores for a 5-fold cross-validation
from sklearn.model_selection import cross_validate

MSE_scores = cross_validate(linreg, X, y, scoring="neg_mean_squared_error", return_train_score=True)
MSE_scores 

{'fit_time': array([0.00810027, 0.00777006, 0.00757909, 0.00790143, 0.00799465]),
 'score_time': array([0.00317073, 0.00292635, 0.00296044, 0.00295448, 0.00306535]),
 'test_score': array([-0.12431546, -0.19350065, -0.1891053 , -0.17079325, -0.20742705]),
 'train_score': array([-0.1717051 , -0.15507936, -0.15659946, -0.16134558, -0.15165049])}

In [10]:
# Get the average MSE score
train_MSE = -(MSE_scores['train_score']).mean()
test_MSE = -(MSE_scores['test_score']).mean()

print('Train Mean Squared Error:', train_MSE)
print('Test Mean Squared Error:', test_MSE)

Train Mean Squared Error: 0.15927599576611776
Test Mean Squared Error: 0.17702834210001112


Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [11]:
# Your answer here
print("""The train_test split method has a train MSE of about 0.1654 and a test MSE of about 0.1523. 
The cross-validation method has a train MSE of 0.1593 and a test MSE of about 0.1770.
I trust the cross-validation method more because 5 models were trained on different sections of the splitted data. 
Therefore, the average MSEs of both the train and test sets cancels out above-average and below-average MSEs
""")

The train_test split method has a train MSE of about 0.1654 and a test MSE of about 0.1523. 
The cross-validation method has a train MSE of 0.1593 and a test MSE of about 0.1770.
I trust the cross-validation method more because 5 models were trained on different sections of the splitted data. 
Therefore, the average MSEs of both the train and test sets cancels out above-average and below-average MSEs



## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [12]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [13]:
def kfolds(data, k):
    folds = []
    # First fold 
    first_fold = data[: (len(data)//k + len(data)%k)]
    folds.append(first_fold)
    
    # From second to last fold
    i=1
    while i <= k:
        fold = data[((len(data)//k)*i + len(data)%k) : ((len(data)//k)*(i+1) + len(data)%k)]
        folds.append(fold)
        i += 1
    return folds

In [14]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 

Empty DataFrame
Columns: [color]
Index: [] 



In [15]:
results[2]

,color
5,indigo
6,violet


### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [16]:
# Apply kfolds() to ames_data with 5 folds
X_folds = kfolds(X, 5)
y_folds = kfolds(y, 5)

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [17]:
# Thought process1 for dataframe concatenation
for n in range(5):
    prog_test = [n-i for i in range(n+1,5)]
    print(prog_test)

[-1, -2, -3, -4]
[-1, -2, -3]
[-1, -2]
[-1]
[]


In [18]:
# Thought process2 for dataframe concatenation
for n in range(5):
    prog_test = [i for i in range(5) if i!=n]
    print(prog_test)

[1, 2, 3, 4]
[0, 2, 3, 4]
[0, 1, 3, 4]
[0, 1, 2, 4]
[0, 1, 2, 3]


In [19]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([X_folds[i] for i in range(5) if i!=n]) # Concatenation
    X_test = X_folds[n]
    y_train = pd.concat([y_folds[i] for i in range(5) if i!=n]) # Concatenation
    y_test = y_folds[n]
    
    # Fit a linear regression model
    linreg = LinearRegression()
    
    # Fit the model on train data
    linreg.fit(X_train, y_train)
    
    # Evaluate test errors
    #train_mse = mean_squared_error(y_train, linreg.predict(X_train))
    test_mse = mean_squared_error(y_test, linreg.predict(X_test))
    test_errs.append(test_mse)

print(test_errs)

[0.1243154614843742, 0.19350064631313132, 0.18910530431311146, 0.1707932525002693, 0.2074270458891694]


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [20]:
# Compare your results with sklearn results
print("""The test-set MSE results obtained from the cross-validation function are compareable to those
produced with cross_val_score method in scikit learn. 
The only difference is that scikit-learn has rounded the errors.""")

The test-set MSE results obtained from the cross-validation function are compareable to those
produced with cross_val_score method in scikit learn. 
The only difference is that scikit-learn has rounded the errors.


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.